In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import torch

from gluonts.dataset.multivariate_grouper import MultivariateGrouper
from gluonts.dataset.repository.datasets import dataset_recipes, get_dataset
from gluonts.evaluation.backtest import make_evaluation_predictions
from gluonts.evaluation import MultivariateEvaluator

from gluonts.torch.model.i_transformer import ITransformerEstimator

/Users/charlesmiller/.local/share/virtualenvs/gluonts-1iNusfKk/lib/python3.9/site-packages/gluonts/json.py:102: UserWarning: Using `json`-module for json-handling. Consider installing one of `orjson`, `ujson` to speed up serialization and deserialization.
  warnings.warn(


In [33]:
from gluonts.dataset.pandas import PandasDataset
url = (
    "https://gist.githubusercontent.com/rsnirwan/a8b424085c9f44ef2598da74ce43e7a3"
    "/raw/b6fdef21fe1f654787fa0493846c546b7f9c4df2/ts_long.csv"
)
df = pd.read_csv(url, index_col=0, parse_dates=True)
df.head()

ds = PandasDataset.from_long_dataframe(df, target="target", item_id="item_id")

In [14]:
df.head()

,target,item_id
2021-01-01 00:00:00,-1.3378,A
2021-01-01 01:00:00,-1.6111,A
2021-01-01 02:00:00,-1.9259,A
2021-01-01 03:00:00,-1.9184,A
2021-01-01 04:00:00,-1.9168,A


In [15]:
df_copy = df.copy()
# Timestamp to drop
timestamp_to_drop = '2021-01-01 03:00:00'
df_copy.drop(pd.Timestamp(timestamp_to_drop), axis=0, inplace=True)
df_copy.head()

,target,item_id
2021-01-01 00:00:00,-1.3378,A
2021-01-01 01:00:00,-1.6111,A
2021-01-01 02:00:00,-1.9259,A
2021-01-01 04:00:00,-1.9168,A
2021-01-01 05:00:00,-1.9681,A


In [16]:
ds_copy = PandasDataset.from_long_dataframe(df_copy, target="target", item_id="item_id")

TypeError: object of type 'NoneType' has no len()

In [38]:
df = pd.read_csv(url, index_col=0, parse_dates=True)
df['timestamp'] = df.index
df.reset_index(drop=True, inplace=True)
df["time_idx"] = df.index
df.drop(columns=["time_stamp"], axis=1,inplace=True)

In [39]:
df

,target,item_id,timestamp,time_idx
0,-1.3378,A,2021-01-01 00:00:00,0
1,-1.6111,A,2021-01-01 01:00:00,1
2,-1.9259,A,2021-01-01 02:00:00,2
3,-1.9184,A,2021-01-01 03:00:00,3
4,-1.9168,A,2021-01-01 04:00:00,4
...,...,...,...,...
2395,1.2349,J,2021-01-10 19:00:00,2395
2396,1.1525,J,2021-01-10 20:00:00,2396
2397,1.1485,J,2021-01-10 21:00:00,2397
2398,1.3248,J,2021-01-10 22:00:00,2398


In [40]:
ds = PandasDataset.from_long_dataframe(df, timestamp="time_idx", target="target", item_id="item_id")

In [48]:
from gluonts.torch import DeepAREstimator
device = torch.device("cpu")

estimator = DeepAREstimator(
    freq='s', 
    prediction_length=24,
    trainer_kwargs={"device": device}
)
predictor = estimator.train(ds)
predictions = predictor.predict(ds)


TypeError: __init__() got an unexpected keyword argument 'device'

In [52]:
import torch
from gluonts.torch import DeepAREstimator
from gluonts.dataset.common import ListDataset
from gluonts.dataset.field_names import FieldName
from gluonts.torch.model.deepar import DeepARLightningModule
from pytorch_lightning import Trainer

# Sample data
training_data = ListDataset(
    [{"start": "2021-01-01 00:00:00", "target": list(range(1, 25))}],
    freq="s"
)

# Set device to CPU
device = torch.device("cpu")

# Define the estimator without specifying the device here
estimator = DeepAREstimator(
    freq='s', 
    prediction_length=24,
    trainer_kwargs={"max_epochs": 10}  # Example, you can adjust as needed
)

# Set device to CPU explicitly in PyTorch Lightning Trainer
trainer = Trainer(
    max_epochs=10,  # Example, you can adjust as needed
    accelerator='cpu'  # Setting accelerator='cpu' forces the use of CPU
)

# Train the model
predictor = estimator.train(training_data, trainer=trainer)

# # Make predictions
# predictions = predictor.predict(training_data)

# # Iterate over predictions and print them
# for forecast in predictions:
#     print(forecast.mean)


GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


NotImplementedError: The operator 'aten::_standard_gamma' is not currently implemented for the MPS device. If you want this op to be added in priority during the prototype phase of this feature, please comment on https://github.com/pytorch/pytorch/issues/77764. As a temporary fix, you can set the environment variable `PYTORCH_ENABLE_MPS_FALLBACK=1` to use the CPU as a fallback for this op. WARNING: this will be slower than running natively on MPS.